In [1]:
import sys; sys.path.append('..')
from osp import *

In [9]:
output_stash = HashStash('osp_slices_1000_nlp')
output_stash


Config,Param,Value
Path,Root Dir,/Users/ryan/.cache/hashstash/osp_slices_1000_nlp
,Filename,data.db
Engine,Engine,lmdb
,Serializer,hashstash
,Compress,lz4
,B64,True
Stats,Len,2505


In [103]:
key,docstr = next(output_stash.items())
doc = stanza.Document.from_serialized(docstr)
word = doc.sentences[1].words[5]
key

'phil/10.2307/20140633__02'

In [86]:
def get_word_context(doc, sent_i, word_i, context_len=2):
    sent = doc.sentences[sent_i]

    prev_context=''
    next_context=''

    words_forward=sent.words[word_i+1:]
    words_backward=reversed(sent.words[:word_i])
    for w in words_forward:
        if len(next_context) < context_len:
            next_context+=w.text+' '
        else:
            break
    for w in words_backward:
        if len(prev_context) < context_len:
            prev_context=w.text+' '+prev_context+' '
        else:
            break

    word = sent.words[word_i]
    out = f'{prev_context.strip()} **{word.text}** {next_context.strip()}'
    out = out.replace('\n',' ').replace(' ,',',').replace(' .', '.').replace(' !','!').replace(' ?','?').replace( ':',':').replace(' ;',';')
    out = out.replace('( ','(').replace('[ ','[').strip().replace(' )',')').replace(' ]',']').replace('"', ' ').replace("'"," ")
    return remove_left_right_punct(out.strip()).strip()

FEAT_N = 100
FEAT_MIN_COUNT = 0

def get_egs(word2count, n = FEAT_N, min_count=FEAT_MIN_COUNT, word2eg={}):
    total = word2count.total()
    o = []
    for w,c in word2count.most_common():
        if n and len(o) >= n:
            break

        c = int(round(c/total*1000))
        if c >= min_count:
            # o.append(f'{w} ({c})')
            eg = word2eg.get(w)
            if eg:
                o.append(f'{w.upper()} ({c}) ["{eg}"]')
            else:
                o.append(f'{w} ({c})')
            n-=1
        else:
            break
        # o.append(w)
    return '; '.join(o)

def get_egs_feat(feat, disc='phil', n=FEAT_N, min_count=FEAT_MIN_COUNT, incl_egs=False):
    feat2word2count = disc2feat2word2count[disc]
    feat2word2eg = disc2feat2word2eg[disc]
    word2count = feat2word2count[feat]
    word2eg = feat2word2eg[feat]
    return get_egs(word2count, n=n, min_count=min_count, word2eg=word2eg if incl_egs else {})


In [95]:
def get_pos_counts(doc, feat2word2count=None, feat2word2eg=None):
    pos_counts = Counter()
    deprel_counts = Counter()
    for sent_i,sent in enumerate(doc.sentences):
        for word_i,word in enumerate(sent.words):
            if word.pos in {'X'} or word.deprel in {'flat'}:
                continue
            pos_counts[word.pos]+=1
            deprel_counts[word.deprel]+=1

            eg_word = word.text.lower()
            if feat2word2count is not None:
                feat2word2count[word.deprel][eg_word]+=1
                feat2word2count[word.pos][eg_word]+=1

            if feat2word2eg is not None:
                eg_context = get_word_context(doc, sent_i, word_i, context_len=context_len).strip()
                feat2word2eg[word.deprel][eg_word] = eg_context
                feat2word2eg[word.pos][eg_word] = eg_context

    sum_pos_counts = sum(pos_counts.values())
    # print(sum_pos_counts)
    pos_counts_rel = {k:int(round(v/sum_pos_counts*1000)) for k,v in pos_counts.items()}

    sum_deprel_counts = sum(deprel_counts.values())
    deprel_counts_rel = {k:int(round(v/sum_deprel_counts*1000)) for k,v in deprel_counts.items()}

    return {**pos_counts_rel,**deprel_counts_rel}



In [102]:
pd.options.display.max_colwidth=None

feat2word2count = defaultdict(Counter)
disc2feat2word2count = {
    'phil': defaultdict(Counter),
    'lit': defaultdict(Counter)
}

disc2feat2word2eg={
    'phil': defaultdict(dict),
    'lit': defaultdict(dict)
}

context_len = 15


id2counts = {}


for id,docstr in tqdm(output_stash.items(),total=len(output_stash)):
    if id not in id2counts:
        doc = stanza.Document.from_serialized(docstr)
        disc = id.split('/')[0]
        feat2word2count = disc2feat2word2count[disc]
        feat2word2eg = disc2feat2word2eg[disc]
        id2counts[id] = get_pos_counts(doc, feat2word2count=feat2word2count, feat2word2eg=feat2word2eg)
    # if len(id2counts) > 100:
        # break

100%|██████████| 2505/2505 [01:24<00:00, 29.49it/s]


In [98]:

df = pd.DataFrame(id2counts).T.rename_axis('id').fillna(0).applymap(int)
df['_target'] = [i.split('/')[0] for i in df.index]
df = df[['_target'] + [c for c in df if c != '_target']]
df

,_target,PRON,AUX,VERB,CCONJ,ADP,PUNCT,NOUN,ADV,PROPN,...,INTJ,reparandum,vocative,advcl:relcl,nmod:unmarked,iobj,csubj:outer,dislocated,orphan,csubj:pass
id,,,,,,,,,,,,,,,,,,,,,
phil/10.2307/20140633__02,phil,89,66,100,34,116,99,190,47,19,...,0,0,0,0,0,0,0,0,0,0
phil/10.2307/43921200__10,phil,8,7,18,46,86,258,225,7,196,...,0,0,0,0,0,0,0,0,0,0
phil/10.2307/26706722__04,phil,38,76,78,21,110,121,215,64,18,...,1,0,0,0,0,0,0,0,0,0
phil/10.2307/2180906__06,phil,46,51,88,30,135,138,199,44,40,...,3,0,0,0,0,0,0,0,0,0
lit/469190__01,lit,62,59,83,38,111,132,211,35,6,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
phil/10.2307/40231597__01,phil,66,65,116,21,101,113,216,56,2,...,0,0,0,2,0,0,0,0,0,0
phil/10.2307/2023490__02,phil,60,81,98,43,109,126,194,38,16,...,0,0,0,0,1,1,0,0,0,0
lit/2872910__05,lit,80,47,102,34,124,129,177,39,29,...,1,0,0,0,3,2,0,2,0,0


In [99]:
pd.options.display.max_rows = 100

df_means = df.groupby('_target').mean()
feat2grp2mean = df_means.to_dict()

ld = []
feat1 = None
for feat in feat2grp2mean:

    feat_d = {'feat':feat}
    grp2mean = feat2grp2mean[feat]
    grp1 = None
    for grp,score in grp2mean.items():
        feat_d[grp+'_egs'] = get_egs_feat(feat, disc=grp, min_count=0, n=50, incl_egs=False)
        feat_d[grp+'_egs2'] = get_egs_feat(feat, disc=grp, n=50, incl_egs=True)

        feat_d[grp]=score

    grps = list(grp2mean.keys())
    if len(grps)>1:
        grp1=grps[0]
        for grp2 in grps[1:]:
            feat_d[f'{grp2}-{grp1}'] = feat_d[grp2] - feat_d[grp1]

    # feat1 = feat
    ld.append(feat_d)
odf = pd.DataFrame(ld).dropna().sort_values('phil-lit',ascending=False).set_index('feat')
for c in odf:
    if not '_egs' in c:
        odf[c] = odf[c].apply(round).apply(int).astype(int)
odf = odf[['phil','lit','phil-lit','phil_egs','lit_egs','phil_egs2','lit_egs2']].sort_values('phil-lit', ascending=False)
# odf[['phil','lit','phil-lit', 'phil_egs']]


In [100]:
# @title Most philosophy
odf.sort_values('phil-lit', ascending=False).head(10)

,phil,lit,phil-lit,phil_egs,lit_egs,phil_egs2,lit_egs2
feat,,,,,,,
AUX,63,48,14,is (325); be (124); are (102); can (65); will (42); does (36); would (34); may (27); have (26); was (23); should (23); do (21); has (21); must (19); could (18); might (16); being (14); were (13); been (11); shall (8); am (5); ought (4); had (4); did (4); need (4),is (293); be (116); are (76); was (70); have (49); may (45); can (42); would (35); has (32); were (29); will (25); does (23); been (21); being (19); might (17); could (16); must (15); should (12); had (12); do (10); did (8); am (7); shall (6); having (4); 's (4),"IS (325) [""research - that **is** a tendency that""]; BE (124) [""is sinful to **be** very much disturbed""]; ARE (102) [""loyal service **are** more pervasive""]; CAN (65) [""which the mind **can** produce complex""]; WILL (42) [""this structure **will** be that of a physical""]; DOES (36) [""set (neither **does** it constitute an""]; WOULD (34) [""of the church **would** limit it to a special""]; MAY (27) [""painting Jones **may** notice the composition""]; HAVE (26) [""Protestant churches **have** been as timid""]; WAS (23) [""modern life as **was** the church of the""]; SHOULD (23) [""experimentation **should** be regarded as""]; DO (21) [""We **do** not here deplore""]; HAS (21) [""within the church **has** become in our own""]; MUST (19) [""aspects of AMR **must** be noted right""]; COULD (18) [""only if it **could** be ordered in some""]; MIGHT (16) [""revolutions - **might** they have been""]; BEING (14) [""physical distinction **being** kept entirely informal""]; WERE (13) [""conceptions of theology **were** originally drawn""]; BEEN (11) [""Protestantism has **been** even less clear""]; SHALL (8) [""I **shall**, however, assume""]; AM (5) [""and if I **am** right, any future""]; OUGHT (4) [""behave as they **ought""]; HAD (4) [""if the church **had** moved with the""]; DID (4) [""Protestantism **did** not materially""]; NEED (4) [""They **need** not be excessively""]","IS (293) [""when the truth **is** restless and under""]; BE (116) [""state that may **be** felt in a fully""]; ARE (76) [""stages at the end **are** also clearly marked""]; WAS (70) [""But he **was** enlarged and filled""]; HAVE (49) [""I **have** already mentioned""]; MAY (45) [""state that **may** be felt in a fully""]; CAN (42) [""parallels that **can** be adduced""]; WOULD (35) [""resemblances one **would** not wish to consider""]; HAS (32) [""story of Cokayne **has** been reinforced""]; WERE (29) [""If he **were** expert enough to""]; WILL (25) [""world of commerce **will** find it difficult""]; DOES (23) [""Does** this mean that""]; BEEN (21) [""of Cokayne has **been** reinforced by a""]; BEING (19) [""more space **being** allotted to the""]; MIGHT (17) [""We **might** expect Ps. The""]; COULD (16) [""see how anyone **could** examine the book""]; MUST (15) [""to Canterbury **must** be regarded as""]; SHOULD (12) [""Indeed, I **should** have thought that""]; HAD (12) [""accounts as they **had** appeared for centuries""]; DO (10) [""of her novels **do** not continually""]; DID (8) [""he nevertheless **did** return though reluctantly""]; AM (7) [""item, not, I **am** sure, within the""]; SHALL (6) [""of God, he **shall** rise""]; HAVING (4) [""attention to Pepper **having** been Lewis student""]; 'S (4) [""you say - it ** s** Morinaga""]"
mark,47,33,14,to (333); that (276); as (72); if (67); of (36); in (28); for (25); whether (25); by (22); because (17); since (11); though (11); while (10); than (9); without (6); although (6); about (6); from (4); so (4); whereas (3); with (3); on (3); once (2); at (2); unless (2),to (364); that (223); as (70); if (54); of (44); because (30); by (30); in (27); for (20); while (19); though (16); whether (11); than (11); before (8); on (7); since (6); without (6); so (6); although (6); from (6); once (5); into (4); unless (3); with (3); wherein (3),"TO (333) [""zealous churchmen **to** criticize the methods""]; THAT (27

In [101]:
# @title Most lit
odf.sort_values('phil-lit', ascending=True).head(5)

,phil,lit,phil-lit,phil_egs,lit_egs,phil_egs2,lit_egs2
feat,,,,,,,
PROPN,21,43,-22,a (20); t (19); husserl (18); smith (17); god (15); kant (14); university (13); spinoza (11); descartes (10); jones (10); kripke (9); cambridge (9); davidson (9); aristotle (9); robin (8); l (7); sally (7); leibniz (7); amr (7); clark (7); press (6); press. (6); a. (6); c (6); schmidt (6),james (14); english (12); england (9); mr. (9); parlement (9); vivaldo (8); rimbaud (8); tophas (8); burney (8); jane (7); valery (7); wynnere (7); sir (7); rousseau (7); literature (7); hogg (7); eric (7); goldsmith (7); god (6); buffon (6); london (6); psalter (6); pocky (5); j. (5); orgilus (5),"A (20) [""simply X did **A** , we may hope""]; T (19) [""indeterminate whether **t** refers to a""]; HUSSERL (18) [""the former by **Husserl**, but it is also""]; SMITH (17) [""appreciate it, while **Smith** merely notices""]; GOD (15) [""through Christ to **God**, were taken over""]; KANT (14) [""concluded, that **Kant** was a Swedenborgian""]; UNIVERSITY (13) [""David E. Taylor **University** of Minnesota Picture""]; SPINOZA (11) [""from Descartes, **Spinoza**, Kant and Bohr""]; DESCARTES (10) [""elements from **Descartes**, Spinoza, Kant""]; JONES (10) [""same painting **Jones** may notice the""]; KRIPKE (9) [""reproduction of it, **Kripke** explicitly denies""]; CAMBRIDGE (9) [""Cambridge : **Cambridge** University Press""]; DAVIDSON (9) [""pump example by **Davidson** et al""]; ARISTOTLE (9) [""entirety, as **Aristotle** comprehended the""]; ROBIN (8) [""state, and that **Robin** will act rationally""]; L (7) [""which sentence of **L** would result in""]; SALLY (7) [""explanation why **Sally** used / despite""]; LEIBNIZ (7) [""Later, **Leibniz** explains how symbolism""]; AMR (7) [""the context of **AMR** is that automata""]; CLARK (7) [""as opposed to **Clark**, does not have""]; PRESS (6) [""Cambridge University **Press""]; PRESS. (6) [""Cambridge University **Press.** Verheij, B. Dialectical""]; A. (6) [""consistently include **A.** G — rdenfors has""]; C (6) [""then action **C**, and so on, just""]; SCHMIDT (6) [""Whereas **Schmidt** emphasizes the""]","JAMES (14) [""Yet **James** was not such an""]; ENGLISH (12) [""specialists in Old **English**, Henry Sweet""]; ENGLAND (9) [""Augustan Age of **England** ] is by Goldsmith""]; MR. (9) [""Mr.** MIuir presumably""]; PARLEMENT (9) [""whereas the **Parlement** poet employes it""]; VIVALDO (8) [""true purpose **Vivaldo** s be nevolence""]; RIMBAUD (8) [""sadistic Prince of **Rimbaud** s fairy - tale""]; TOPHAS (8) [""and fancy as **Tophas""]; BURNEY (8) [""Burney** satirizes a scenario""]; JANE (7) [""suggest that **Jane** Austen is being""]; VALERY (7) [""terminology used by **Valery** with that used""]; WYNNERE (7) [""The **Wynnere** poet, however""]; SIR (7) [""the relief of **Sir** John Moore s army""]; ROUSSEAU (7) [""lanceur of **Rousseau""]; LITERATURE (7) [""German V (Modem **Literature""]; HOGG (7) [""Hogg** s achievement""]; ERIC (7) [""suggested by **Eric** — as a risky""]; GOLDSMITH (7) [""not point to **Goldsmith""]; GOD (6) [""the power of **God**, he shall rise""]; BUFFON (6) [""headlines that **Buffon** had been the""]; LONDON (6) [""Luce of The **London** Prodigal plays""]; PSALTER (6) [""beginning of the **Psalter** was really like""]; POCKY (5) [""and the milk **Pocky""]; J. (5) [""J.** A. Without the""]; ORGILUS (5) [""Orgilus** s line of life""]"
case,104,123,-19,of (339); in (169); to (87); for (53); with (44); as (36); by (34); 's (33); on (32); from (28); at (25); about (15); between (12); into (8); than (7); within (7); like (6); upon (4); under (4); according (4); without (3); over (3); after (3); ' (3); against (3),of (343); in (162); to (75); 's (54); for (47); as (45); with (42); by (37); from (33); on (31); at (23); between (11); into (10); about (9); than (8); like (7); through (6); among (4); without (4); within (4); upon (4); over (3); against (3); out (3); under (3),"OF (339) [""